<a href="https://colab.research.google.com/github/Rumeysakeskin/Question-Answering-BERT/blob/main/question_answering_bert_squad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTALL NEMO

In [1]:
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[nlp]

DEPRECATION: git+https://github.com/NVIDIA/NeMo.git@main#egg=nemo_toolkit[nlp] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/NVIDIA/NeMo.git (to revision main) to /tmp/pip-install-ks0i101a/nemo-toolkit_2b54be33f8af4c438492e4f8c3b504f8
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/NeMo.git /tmp/pip-install-ks0i101a/nemo-toolkit_2b54be33f8af4c438492e4f8c3b504f8
  Resolved https://github.com/NVIDIA/NeMo.git to commit ecc116c29a14534abfe393f6eee4339b24ca6e57
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import wget
import gc

import pytorch_lightning as pl
from omegaconf import OmegaConf

from nemo.collections.nlp.models.question_answering.qa_bert_model import BERTQAModel
from nemo.utils.exp_manager import exp_manager

pl.seed_everything(42)
gc.disable()

[NeMo W 2023-04-11 10:21:19 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-04-11 10:21:30 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-04-11 10:21:30 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.
INFO:lightning_fabric.utilities.seed:Global seed set to 42


In [3]:
# set the following paths
DATA_DIR = "data_dir" # directory for storing datasets
WORK_DIR = "work_dir" # directory for storing trained models, logs, additionally downloaded scripts

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(WORK_DIR, exist_ok=True)

In [4]:
# download the model's default configuration file 
config_dir = WORK_DIR + '/conf/'
os.makedirs(config_dir, exist_ok=True)
if not os.path.exists(config_dir + "qa_conf.yaml"):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/question_answering/conf/qa_conf.yaml', config_dir)
else:
    print ('config file already exists')

# this will print the entire default config of the model
config_path = f'{WORK_DIR}/conf/qa_conf.yaml'
# print(config_path)
config = OmegaConf.load(config_path)
# print("Default Config - \n")
# print(OmegaConf.to_yaml(config))

DOWNLOAD AND PREPROCESS SQUAD DATASET

In [5]:
# download get_squad.py script to download and preprocess the SQuAD data
os.makedirs(WORK_DIR, exist_ok=True)
if not os.path.exists(WORK_DIR + '/get_squad.py'):
    print('Downloading get_squad.py...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/question_answering/get_squad.py', WORK_DIR)
else:
    print ('get_squad.py already exists')

In [6]:
# download and preprocess the data
!python $WORK_DIR/get_squad.py --destDir $DATA_DIR
!ls -LR {DATA_DIR}/squad

[NeMo I 2023-04-11 10:21:36 get_squad:66] data_dir
[NeMo I 2023-04-11 10:21:36 get_squad:47] Downloading: https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
[NeMo I 2023-04-11 10:21:38 get_squad:47] Downloading: https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
[NeMo I 2023-04-11 10:21:38 get_squad:47] Downloading: https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
[NeMo I 2023-04-11 10:21:41 get_squad:47] Downloading: https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
data_dir/squad:
v1.1  v2.0

data_dir/squad/v1.1:
dev-v1.1.json  train-v1.1.json

data_dir/squad/v2.0:
dev-v2.0.json  train-v2.0.json


SET DATASET CONFIG VALUES

In [7]:
# if True, model will load features from cache if file is present, or
# create features and dump to cache file if not already present
config.model.dataset.use_cache = False

# indicates whether the dataset has unanswerable questions
config.model.dataset.version_2_with_negative = True

# indicates whether the dataset is of extractive nature or not
# if True, context spans/chunks that do not contain answer are treated as unanswerable 
config.model.dataset.check_if_answer_in_context = True

# set file paths for train, validation, and test datasets
config.model.train_ds.file = f"{DATA_DIR}/squad/v2.0/train-v2.0.json"
config.model.validation_ds.file = f"{DATA_DIR}/squad/v2.0/dev-v2.0.json"
config.model.test_ds.file = f"{DATA_DIR}/squad/v2.0/dev-v2.0.json"

# set batch sizes for train, validation, and test datasets
config.model.train_ds.batch_size = 8
config.model.validation_ds.batch_size = 8
config.model.test_ds.batch_size = 8

# set number of samples to be used from dataset. setting to -1 uses entire dataset
config.model.train_ds.num_samples = 5000
config.model.validation_ds.num_samples = 1000
config.model.test_ds.num_samples = 100

SET TRAINER CONFIG VALUES

In [16]:
config.trainer.max_epochs = 5
config.trainer.max_steps = -1 # takes precedence over max_epochs
config.trainer.precision = 16
config.trainer.devices = [0] # 0 for CPU, or list of the GPUs to use [0] this tutorial does not support multiple GPUs. If needed please use NeMo/examples/nlp/question_answering/question_answering.py
config.trainer.accelerator = "gpu"
config.trainer.strategy="dp"

SET EXPERIMENT MANAGER CONFIG VALUES

In [17]:
config.exp_manager.exp_dir = WORK_DIR
config.exp_manager.name = "QA-SQuAD2"
config.exp_manager.create_wandb_logger=False

TRAINING WITH BERT MODEL 

In [18]:
# set language model and tokenizer to be used
# tokenizer is derived from model if a tokenizer name is not provided
config.model.language_model.pretrained_model_name = "bert-base-uncased"
config.model.tokenizer.tokenizer_name = "bert-base-uncased"

# path where model will be saved
config.model.nemo_path = f"{WORK_DIR}/checkpoints/bert_squad_v2_0.nemo"

config.exp_manager.create_checkpoint_callback = True

config.model.optim.lr = 3e-5

In [19]:
trainer = pl.Trainer(**config.trainer)
model = BERTQAModel(config.model, trainer=trainer)
trainer.fit(model)
trainer.test(model)

model.save_to(config.model.nemo_path)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit None Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


[NeMo I 2023-04-11 10:30:26 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-base-uncased, vocab_file: /root/.cache/huggingface/nemo_nlp_tmp/ae0d012864bdb2474ba67537c3f5e0fa/vocab.txt, merges_files: None, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
[NeMo W 2023-04-11 10:30:27 modelPT:245] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.


[NeMo I 2023-04-11 10:30:27 qa_processing:106] mean no. of chars in doc: 839.2727272727273
[NeMo I 2023-04-11 10:30:27 qa_processing:107] max no. of chars in doc: 1895
[NeMo I 2023-04-11 10:30:27 qa_processing:106] mean no. of chars in doc: 677.5487804878048
[NeMo I 2023-04-11 10:30:27 qa_processing:107] max no. of chars in doc: 1782
[NeMo I 2023-04-11 10:30:27 qa_processing:106] mean no. of chars in doc: 828.0972222222222
[NeMo I 2023-04-11 10:30:27 qa_processing:107] max no. of chars in doc: 2132
[NeMo I 2023-04-11 10:30:27 qa_processing:106] mean no. of chars in doc: 540.0
[NeMo I 2023-04-11 10:30:27 qa_processing:107] max no. of chars in doc: 1423
[NeMo I 2023-04-11 10:30:27 qa_processing:106] mean no. of chars in doc: 756.71875
[NeMo I 2023-04-11 10:30:27 qa_processing:107] max no. of chars in doc: 1747
[NeMo I 2023-04-11 10:30:27 qa_processing:106] mean no. of chars in doc: 732.4418604651163
[NeMo I 2023-04-11 10:30:27 qa_processing:107] max no. of chars in doc: 3076
[NeMo I 2023

  0%|          | 0/5000 [00:00<?, ?it/s]

...

[NeMo I 2023-04-11 10:49:43 qa_bert_model:140] val exact: 29.1
[NeMo I 2023-04-11 10:49:43 qa_bert_model:140] val f1: 34.226934976959754
[NeMo I 2023-04-11 10:49:43 qa_bert_model:140] val total: 1000.0
[NeMo I 2023-04-11 10:49:43 qa_bert_model:140] val HasAns_exact: 58.433734939759034
[NeMo I 2023-04-11 10:49:43 qa_bert_model:140] val HasAns_f1: 68.7287850942967
[NeMo I 2023-04-11 10:49:43 qa_bert_model:140] val HasAns_total: 498.0
[NeMo I 2023-04-11 10:49:43 qa_bert_model:140] val NoAns_exact: 0.0
[NeMo I 2023-04-11 10:49:43 qa_bert_model:140] val NoAns_f1: 0.0
[NeMo I 2023-04-11 10:49:43 qa_bert_model:140] val NoAns_total: 502.0


INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

[NeMo I 2023-04-11 10:49:47 qa_bert_model:140] test exact: 32.0
[NeMo I 2023-04-11 10:49:47 qa_bert_model:140] test f1: 35.30173992673992
[NeMo I 2023-04-11 10:49:47 qa_bert_model:140] test total: 100.0
[NeMo I 2023-04-11 10:49:47 qa_bert_model:140] test HasAns_exact: 71.11111111111111
[NeMo I 2023-04-11 10:49:47 qa_bert_model:140] test HasAns_f1: 78.44831094831093
[NeMo I 2023-04-11 10:49:47 qa_bert_model:140] test HasAns_total: 45.0
[NeMo I 2023-04-11 10:49:47 qa_bert_model:140] test NoAns_exact: 0.0
[NeMo I 2023-04-11 10:49:47 qa_bert_model:140] test NoAns_f1: 0.0
[NeMo I 2023-04-11 10:49:47 qa_bert_model:140] test NoAns_total: 55.0


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│     test_HasAns_exact     │     71.11111450195312     │
│      test_HasAns_f1       │     78.44831085205078     │
│     test_HasAns_total     │           45.0            │
│     test_NoAns_exact      │            0.0            │
│       test_NoAns_f1       │            0.0            │
│     test_NoAns_total      │           55.0            │
│        test_exact         │           32.0            │
│          test_f1          │     35.30173873901367     │
│         test_loss         │     7.904482841491699     │
│        test_total         │           100.0           │
└───────────────────────────┴───────────────────────────┘

LOAD SAVED MODEL

In [21]:
model = BERTQAModel.restore_from(config.model.nemo_path)

eval_device = [config.trainer.devices[0]] if isinstance(config.trainer.devices, list) else 1
model.trainer = pl.Trainer(
    devices=eval_device,
    accelerator=config.trainer.accelerator,
    precision=16,
    logger=False,
)
config.exp_manager.create_checkpoint_callback = False
exp_dir = exp_manager(model.trainer, config.exp_manager)

[NeMo I 2023-04-11 11:06:26 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-base-uncased, vocab_file: /tmp/tmpkvx5jae3/5df292155e1b4d28aa5bc68ffe51b6d4_vocab.txt, merges_files: None, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
[NeMo W 2023-04-11 11:06:26 modelPT:245] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.
[NeMo W 2023-04-11 11:06:26 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    file: data_dir/squad/v2.0/train-v2.0.json
    batch_size: 8
    shuffle: true
    num_samples: 5000
    num_workers: 1
    drop_last: false
    pin_memory: false
    
[NeMo W 2023-04-11 11:06:26 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    file: data_dir/squad/v2.0/dev-v2.0.json
    batch_size: 8
    shuffle: fa

[NeMo I 2023-04-11 11:06:30 save_restore_connector:249] Model BERTQAModel was successfully restored from /content/work_dir/checkpoints/bert_squad_v2_0.nemo.


INFO:pytorch_lightning.utilities.rank_zero:Using 16bit None Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


[NeMo I 2023-04-11 11:06:30 exp_manager:374] Experiments will be logged at work_dir/QA-SQuAD2/2023-04-11_10-26-08
[NeMo I 2023-04-11 11:06:30 exp_manager:797] TensorboardLogger has been set up


INFERENCE

In [36]:
import json

def dump_json(filepath, data):
    with open(filepath, "w") as f:
        json.dump(data, f)

def create_inference_data_format(context, question):

  squad_data = {"data": [{"title": "inference", "paragraphs": []}], "version": "v2.1"}
  squad_data["data"][0]["paragraphs"].append(
            {
                "context": context,
                "qas": [
                    {"id": 0, "question": question,}
                ],
            }
        )
  return squad_data

context = "The Amazon rainforest is a moist broadleaf forest that covers most of the Amazon basin of South America. This basin encompasses 7,000,000 square kilometres (2,700,000 sq mi), of which 5,500,000 square kilometres (2,100,000 sq mi) are covered by the rainforest. The majority of the forest is contained within Brazil, with 60% of the rainforest, followed by Peru with 13%, and Colombia with 10%."

question = "Which country has the most?"

inference_filepath = "inference.json"

inference_data = create_inference_data_format(context, question)
dump_json(inference_filepath, inference_data)

predictions = model.inference("inference.json")
question = predictions[1][0][0]["question"]
answer = predictions[1][0][0]["text"]
probability = predictions[1][0][0]["probability"]

print(f"\n> Question: {question}\n> Answer: {answer}\n Probability: {probability}")

100%|██████████| 1/1 [00:00<00:00, 8112.77it/s]



> Question: Which country has the most?
> Answer: Brazil
 Probability: 0.9688649039578262
